In [29]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='pnc', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pnc...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [39]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [31]:
item_meta = mysql.read_table(spark, "metainfo", "meta_vc_sales_items")

In [32]:
item_meta.createOrReplaceTempView("item_meta")

In [33]:
pcs4_ep_q = """
    select product_id
    from item_meta
    where platform = "STEAM" and event_name like '%pcs4%' and currency = "esportspoint"
"""
pcs4_ep = spark.sql(pcs4_ep_q).toPandas()["product_id"].tolist()

pcs5_ep_q = """
    select product_id
    from item_meta
    where platform = "STEAM" and event_name like '%pcs5%' and currency = "esportspoint"
"""
pcs5_ep = spark.sql(pcs5_ep_q).toPandas()["product_id"].tolist()

pcs6_ep_q = """
    select product_id
    from item_meta
    where platform = "STEAM" and event_name like '%pcs6%' and currency = "esportspoint"
"""
pcs6_ep = spark.sql(pcs6_ep_q).toPandas()["product_id"].tolist()

In [7]:
# pcs4_pc = pre_load_data_mart("pc", "2021-06-02", "2021-06-11", "gcoin_use").where(lower(col("event_name")).like("%pcs%")).groupBy("account_id").agg(sum("price").alias("total_use"))
pcs4_console = pre_load_data_mart("console", "2021-06-10", "2021-06-23", "gcoin_use").where(lower(col("event_name")).like("%pcs%"))\
    .withColumn("ep", when(col("product_id").isin(pcs4_ep), "ep").otherwise("no_ep"))\
    .groupBy("ep", "account_id").agg(sum("price").alias("total_use"))

In [8]:
# pcs5_pc = load_data_mart("pc", "2021-09-08", "2021-09-17", "gcoin_use").where(lower(col("event_name")).like("%pcs%")).groupBy("account_id").agg(sum("price").alias("total_use"))
pcs5_console = load_data_mart("console", "2021-09-16", "2021-09-29", "gcoin_use").where(lower(col("event_name")).like("%pcs%"))\
    .withColumn("ep", when(col("product_id").isin(pcs5_ep), "ep").otherwise("no_ep"))\
    .groupBy("ep", "account_id").agg(sum("price").alias("total_use"))

In [9]:
# pcs6_pc = load_data_mart("pc", "2022-03-16", "2022-03-25", "gcoin_use").where(lower(col("event_name")).like("%pcs%")).groupBy("account_id").agg(sum("price").alias("total_use"))
pcs6_console = load_data_mart("console", "2022-03-24", "2022-04-06", "gcoin_use").where(lower(col("event_name")).like("%pcs%"))\
    .withColumn("ep", when(col("product_id").isin(pcs6_ep), "ep").otherwise("no_ep"))\
    .groupBy("ep", "account_id").agg(sum("price").alias("total_use"))

In [10]:
# pcs6_pc_membership = load_membership("pubg", "pc", "2022-03-25", "2022-03-25").join(pcs6_pc, "account_id")
pcs6_console_membership = load_membership("pubg", "console", "2022-04-06", "2022-04-06").join(pcs6_console, "account_id")

### PCS4

In [11]:
pcs4_pc.where(col("total_use") >= 1400).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|            20661|
+-----------------+



In [11]:
pcs4_console.where((col("ep") == "no_ep") & (col("total_use") >= 1400)).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             1151|
+-----------------+



In [12]:
pcs4_console.where(col("ep") == "ep").select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             5035|
+-----------------+



### PCS5

In [13]:
pcs5_pc.where(col("total_use") >= 1400).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|            72529|
+-----------------+



In [13]:
pcs5_console.where((col("ep") == "no_ep") & (col("total_use") >= 1400)).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             2492|
+-----------------+



In [14]:
pcs5_console.where(col("ep") == "ep").select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             5983|
+-----------------+



### PCS6

In [15]:
pcs6_pc.where(col("total_use") >= 1400).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|            25671|
+-----------------+



In [15]:
pcs6_console.where((col("ep") == "no_ep") & (col("total_use") >= 1400)).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             2383|
+-----------------+



In [16]:
pcs6_console.where(col("ep") == "ep").select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             4268|
+-----------------+



In [23]:
pcs6_pc_membership.where((col('total_use') >= 1400) & (col("membership").isin(["plus", "basic"]))).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             2980|
+-----------------+



In [17]:
pcs6_console_membership.where((col("ep") == "no_ep") & (col('total_use') >= 1400) & (col("membership").isin(["plus", "basic"]))).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|              429|
+-----------------+



In [18]:
pcs6_console_membership.where((col("ep") == "ep") & (col("membership").isin(["plus", "basic"]))).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|              723|
+-----------------+



In [17]:
pgc_pc = load_data_mart("pc", "2021-11-12", "2021-11-21", "gcoin_use").where(lower(col("event_name")).like("%pgc%")).groupBy("account_id").agg(sum("price").alias("total_use"))
pgc_console = load_data_mart("console", "2021-11-12", "2021-11-21", "gcoin_use").where(lower(col("event_name")).like("%pgc%")).groupBy("account_id").agg(sum("price").alias("total_use"))

In [18]:
pgc_pc.where(col("total_use") >= 1400).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|           187710|
+-----------------+



In [19]:
pgc_console.where(col("total_use") >= 1400).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|             5216|
+-----------------+



In [25]:
pgc_pc_m416_buyer = load_data_mart("pc", "2021-11-12", "2021-11-21", "gcoin_use").where((lower(col("event_name")).like("%pgc%")) & (lower(col('product_name')).like("%m416%"))).select("account_id").distinct()
pgc_console_m416_buyer = load_data_mart("console", "2021-11-12", "2021-11-21", "gcoin_use").where((lower(col("event_name")).like("%pgc%")) & (lower(col('product_name')).like("%m416%"))).select("account_id").distinct()

In [29]:
pgc_pc_m416_buyer.collect()

[]

In [27]:
pgc_pc.where((col("total_use") >= 1400) & (col("account_id").isin(pgc_pc_m416_buyer.collect()))).select(count("account_id")).show()

+-----------------+
|count(account_id)|
+-----------------+
|                0|
+-----------------+

